# Dropout
:label:`sec_dropout`


Let's think briefly about what we
expect from a good predictive model.
We want it to peform well on unseen data.
Classical generalization theory
suggests that to close the gap between
train and test performance,
we should aim for a simple model.
Simplicity can come in the form
of a small number of dimensions.
We explored this when discussing the
monomial basis functions of linear models
in :numref:`sec_generalization_basics`.
Additionally, as we saw when discussing weight decay
($\ell_2$ regularization) in :numref:`sec_weight_decay`,
the (inverse) norm of the parameters also
represents a useful measure of simplicity.
Another useful notion of simplicity is smoothness,
i.e., that the function should not be sensitive
to small changes to its inputs.
For instance, when we classify images,
we would expect that adding some random noise
to the pixels should be mostly harmless.

:citet:`Bishop.1995` formalized
this idea when he proved that training with input noise
is equivalent to Tikhonov regularization.
This work drew a clear mathematical connection
between the requirement that a function be smooth (and thus simple),
and the requirement that it be resilient
to perturbations in the input.

Then, :citet:`Srivastava.Hinton.Krizhevsky.ea.2014`
developed a clever idea for how to apply Bishop's idea
to the internal layers of a network, too.
Their idea, called *dropout*, involves
injecting noise while computing
each internal layer during forward propagation,
and it has become a standard technique
for training neural networks.
The method is called *dropout* because we literally
*drop out* some neurons during training.
Throughout training, on each iteration,
standard dropout consists of zeroing out
some fraction of the nodes in each layer
before calculating the subsequent layer.

To be clear, we are imposing
our own narrative with the link to Bishop.
The original paper on dropout
offers intuition through a surprising
analogy to sexual reproduction.
The authors argue that neural network overfitting
is characterized by a state in which
each layer relies on a specific
pattern of activations in the previous layer,
calling this condition *co-adaptation*.
Dropout, they claim, breaks up co-adaptation
just as sexual reproduction is argued to
break up co-adapted genes.
While such an justification of this theory is certainly up for debate,
the dropout technique itself has proved enduring,
and various forms of dropout are implemented
in most deep learning libraries. 


The key challenge is how to inject this noise.
One idea is to inject it in an *unbiased* manner
so that the expected value of each layer---while fixing
the others---equals the value it would have taken absent noise.
In Bishop's work, he added Gaussian noise
to the inputs to a linear model.
At each training iteration, he added noise
sampled from a distribution with mean zero
$\epsilon \sim \mathcal{N}(0,\sigma^2)$ to the input $\mathbf{x}$,
yielding a perturbed point $\mathbf{x}' = \mathbf{x} + \epsilon$.
In expectation, $E[\mathbf{x}'] = \mathbf{x}$.

In standard dropout regularization,
one zeros out some fraction of the nodes in each layer
and then *debiases* each layer by normalizing
by the fraction of nodes that were retained (not dropped out).
In other words,
with *dropout probability* $p$,
each intermediate activation $h$ is replaced by
a random variable $h'$ as follows:

$$
\begin{aligned}
h' =
\begin{cases}
    0 & \textrm{ with probability } p \\
    \frac{h}{1-p} & \textrm{ otherwise}
\end{cases}
\end{aligned}
$$

By design, the expectation remains unchanged, i.e., $E[h'] = h$.

In [12]:
using Pkg
Pkg.activate("../../d2lai")
using d2lai, Flux, Distributions

  Activating project at `/workspace/workspace/d2l-julia/d2lai`


## Dropout in Practice

Recall the MLP with a hidden layer and five hidden units
from :numref:`fig_mlp`.
When we apply dropout to a hidden layer,
zeroing out each hidden unit with probability $p$,
the result can be viewed as a network
containing only a subset of the original neurons.
In :numref:`fig_dropout2`, $h_2$ and $h_5$ are removed.
Consequently, the calculation of the outputs
no longer depends on $h_2$ or $h_5$
and their respective gradient also vanishes
when performing backpropagation.
In this way, the calculation of the output layer
cannot be overly dependent on any
one element of $h_1, \ldots, h_5$.

![MLP before and after dropout.](../img/dropout2.svg)
:label:`fig_dropout2`

Typically, we disable dropout at test time.
Given a trained model and a new example,
we do not drop out any nodes
and thus do not need to normalize.
However, there are some exceptions:
some researchers use dropout at test time as a heuristic
for estimating the *uncertainty* of neural network predictions:
if the predictions agree across many different dropout outputs,
then we might say that the network is more confident.

## Implementation from Scratch

To implement the dropout function for a single layer,
we must draw as many samples
from a Bernoulli (binary) random variable
as our layer has dimensions,
where the random variable takes value $1$ (keep)
with probability $1-p$ and $0$ (drop) with probability $p$.
One easy way to implement this is to first draw samples
from the uniform distribution $U[0, 1]$.
Then we can keep those nodes for which the corresponding
sample is greater than $p$, dropping the rest.

In the following code, we (**implement a `dropout_layer` function
that drops out the elements in the tensor input `X`
with probability `dropout`**),
rescaling the remainder as described above:
dividing the survivors by `1.0-dropout`.

In [3]:
function dropout_layer(X::AbstractArray, dropout)
    probs = rand(Bernoulli(dropout), size(X, 1))
    return probs .* X
end


dropout_layer (generic function with 1 method)

We can test out the dropout_layer function on a few examples. In the following lines of code, we pass our input X through the dropout operation, with probabilities 0, 0.5, and 1, respectively.

In [6]:
X = reshape(1:16, 2, 8)
println("dropout with p = 0", dropout(X, 0.))
println("dropout with p = 0.5", dropout(X, 0.5))
println("dropout with p = 1.0", dropout(X, 1.))

dropout with p = 0[1.0 3.0 5.0 7.0 9.0 11.0 13.0 15.0; 2.0 4.0 6.0 8.0 10.0 12.0 14.0 16.0]
dropout with p = 0.5[2.0 0.0 0.0 14.0 18.0 0.0 0.0 0.0; 0.0 8.0 12.0 0.0 0.0 24.0 28.0 32.0]
dropout with p = 1.0[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]


### Defining the Model

The model below applies dropout to the output
of each hidden layer (following the activation function).
We can set dropout probabilities for each layer separately.
A common choice is to set
a lower dropout probability closer to the input layer.
We ensure that dropout is only active during training.


In [16]:
mutable struct DropoutScratchMLP{N, A} <: AbstractClassifier
    net::N 
    args::A 
    train::Bool
end
Flux.@layer DropoutScratchMLP trainable=(net,)
function DropoutScratchMLP(; args...)
    net = Chain(Dense(args[:num_inputs], args[:num_hidden_1]), Dense(args[:num_hidden_1], args[:num_hidden_2]), Dense(args[:num_hidden_2], args[:num_outputs]), Flux.softmax)
    DropoutScratchMLP(net, NamedTuple(args), true)
end

function d2lai.forward(mlp::DropoutScratchMLP, x)
    lin1, lin2, lin3, softmax = mlp.net.layers
    h1 = model.train ? dropout_layer(lin1(x), mlp.args.dropout_1) : lin1(x)
    h2 = model.train ? dropout_layer(lin2(h1), mlp.args.dropout_2) : lin2(h1)
    h3 = lin3(h2)
    return softmax(h3)
end


### Training

The following is similar to the training of MLPs described previously.


In [ ]:
hparams = (num_inputs = 28*28, num_outputs = 10, num_hidden_1 = 256, num_hidden_2 = 256,
           dropout_1 = 0.5, dropout_2 = 0.5, lr = 0.1)
model = DropoutScratchMLP(; hparams...)

opt = Descent(0.1)
data = d2lai.FashionMNISTData(; batchsize = 256, flatten = true)
trainer = Trainer(model, data, opt; max_epochs = 10)
d2lai.fit(trainer)